In [0]:
import keras
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lxml
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob

In [0]:
class Scrape:
    def __init__(self):
        self.url="https://tiny.cc/sck34y"
        self.reviews=[]
        self.labels=[]
        self.positive=[]
        self.negative=[]
        self.neutral=[]
    
    def get_reviews(self,s,url):
        s.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
        response = s.get(url,headers=s.headers)
        soup = BeautifulSoup(response.text,"lxml")
        return soup.find_all("span",{"data-hook":"review-body"})

    def get_tag(self,s,url):   #pagination to scrape 100 reviews
        s.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
        response = s.get(url,headers=s.headers)
        soup = BeautifulSoup(response.text,"lxml")
      
        try:
            link=soup.find("li",{"class":"a-last"})
            a=link.find('a')
            b="https://www.amazon.in"+a['href']
        
            return b
        except:
            pass 

    def sentiment(self,text):    # sentiment analysis

        score =TextBlob(text).sentiment.polarity
        if(score==0):
            self.neutral.append(text)
            return "neutral"
        elif(score>0):
            self.positive.append(text)
            return "positive"
        else:
            self.negative.append(text)
            return "negative" 

    def review_(self):   #Fectchimg the reviews
        link = 'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
        i=0
        with requests.Session() as s:
            while(i <=100):
                if(i>=100):
                    break
                try:
                    
                    for review in self.get_reviews(s,link):
                        
                        #self.reviews.append(review)    
#                         print(i,")",f'{review.text}\n')
                          text=str(review.text)
#                         self.reviews[str(i)]=text
                          t2=self.sentiment(text)
#                         print("Analysis:",t2,"\n")
                          self.reviews.append(text)
                          self.labels.append(t2)
                          
                            
                          i=i+1
                except UnicodeEncodeError:
                    pass     
                    
                link=self.get_tag(s,link)  


In [0]:
s=Scrape()
s.review_()

In [0]:
tokenizer = Tokenizer(num_words=5000, split=" ")
tokenizer.fit_on_texts(s.reviews)

X = tokenizer.texts_to_sequences(s.reviews)
X = pad_sequences(X,maxlen=88)

In [0]:
model = Sequential()
model.add(Embedding(5000, 128))
#model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(50, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y = pd.get_dummies(s.labels).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# meaning [ 0,0,1]= positive
# [0,1,0]= neutral
# [1,0,0]=  negative

In [39]:

batch_size = int(len(X_train)/10)
epochs = 10

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)

predictions = model.predict(X_test)

Epoch 1/10
 - 6s - loss: 1.0768 - acc: 0.6000
Epoch 2/10
 - 2s - loss: 0.9544 - acc: 0.6250
Epoch 3/10
 - 2s - loss: 0.7853 - acc: 0.6250
Epoch 4/10
 - 2s - loss: 0.6195 - acc: 0.6875
Epoch 5/10
 - 2s - loss: 0.5111 - acc: 0.7625
Epoch 6/10
 - 2s - loss: 0.2790 - acc: 0.9250
Epoch 7/10
 - 2s - loss: 0.5929 - acc: 0.8625
Epoch 8/10
 - 2s - loss: 0.4098 - acc: 0.8750
Epoch 9/10
 - 2s - loss: 0.2682 - acc: 0.8875
Epoch 10/10
 - 2s - loss: 0.2086 - acc: 0.9500


In [41]:
score,acc= model.evaluate(X_test,y_test,batch_size=batch_size, verbose=2)
print(score)
print("Accuracy :",acc)

0.529214608669281
Accuracy : 0.7
